<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_training%2C_Entity_Ruler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This example demonstrates how to use rules-augmented model-based NER to locate stock codes

In [1]:
!pip install --quiet -U spacy
!python -m spacy download zh_core_web_lg
!git clone http://github.com/raymondwcs/learning_spacy

     |████████████████████████████████| 5.9 MB 5.2 MB/s 
     |████████████████████████████████| 10.1 MB 43.6 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 456 kB 71.3 MB/s 
     |████████████████████████████████| 623 kB 37.4 MB/s 
     |████████████████████████████████| 603.8 MB 9.1 kB/s 
     |████████████████████████████████| 2.4 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')
Cloning into 'learning_spacy'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 228 (delta 129), reused 57 (delta 19), pack-reused 0
Receiving objects: 100% (228/228), 66.24 MiB | 24.66 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [2]:
import spacy
import random
from spacy.training.example import Example
from spacy import displacy


In [3]:
nlp = spacy.load('zh_core_web_lg')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")

# Part 1 - Train model-based NER

## Data for training the model-based NER
Training can also be done via CLI.  Details below.

https://github.com/raymondwcs/learning_spacy/tree/main/NER_Training_CLI

In [4]:
TRAIN_DATA = [
    ("一齊係國企成份股調整期間 大家一齊不問價掃貨 尤其是386 857 成份股佔指數比重只係5%左右 冇咩受新計法影響 挾死班歐美鬼佬 港股10月29000 遠必挾之", 
        {"entities": [(26,29,"STOCK"),(30,33,"STOCK")]}),
    ("高盛：維持對騰訊(0700)買入評級 目標價705港元高盛發表報告指,與騰訊管理層於路演活動溝通後,重申對騰訊嘅積極正面睇法。",
        {"entities": [(6,8,"STOCK")]}),
    ("中電、匯控、恒大8月暴升8.5%。",
        {'entities': [(0,2,"STOCK"),(3,5,"STOCK"),(6,8,"STOCK")]}),
    ("匯控(00005)將於下周一（2日）公布2021年度中期業績。",
        {'entities': [(0,2,"STOCK")]}),
    ("中電，匯控，港交所齊齊跌8.5%金融海嘯後最差。",
        {"entities": [(0,2,"STOCK"),(3,5,"STOCK"),(6,9,"STOCK")]}),    
    ("港燈將恢復派息",
        {"entities": [(0,2,"STOCK")]})
]

In [5]:
ner=nlp.get_pipe("ner")

# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

## Start training

In [6]:
epoch = 50
optimizer = nlp.resume_training()

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = spacy.util.minibatch(TRAIN_DATA,size=2)
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.2,sgd=optimizer,losses=losses)
    print("Losses", losses)

Losses {'ner': 24.032677430134182}
Losses {'ner': 20.957658372942543}
Losses {'ner': 17.8215304177172}
Losses {'ner': 16.057177198412358}
Losses {'ner': 13.3660877659215}
Losses {'ner': 12.769929269395245}
Losses {'ner': 9.075146013772947}
Losses {'ner': 5.497757006217853}
Losses {'ner': 4.3064410333596825}
Losses {'ner': 2.5535572147517485}
Losses {'ner': 4.9149026682048}
Losses {'ner': 0.3275314181357267}
Losses {'ner': 0.09761495143317715}
Losses {'ner': 0.02720043247906884}
Losses {'ner': 0.3503842842038852}
Losses {'ner': 0.017482860077863777}
Losses {'ner': 0.0010997636396456172}
Losses {'ner': 0.0041796161211071145}
Losses {'ner': 7.643509672759397e-05}
Losses {'ner': 2.5528735193971694e-05}
Losses {'ner': 0.00013847413352526471}
Losses {'ner': 7.197907113176314e-08}
Losses {'ner': 2.258532660834581e-06}
Losses {'ner': 7.55098562942737e-05}
Losses {'ner': 1.65775596156289e-05}
Losses {'ner': 2.8982274757333573e-08}
Losses {'ner': 4.662615324543495e-05}
Losses {'ner': 1.305861935

## Debug

In [7]:
for text, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    # print(text,ent)
    # print(text[ent[0]:ent[1]])
    doc = nlp(text)
    char_span = doc.char_span(ent[0],ent[1])
    if char_span is None:  # start and end don't map to tokens
        print("Misaligned tokens", text, ent)

## Save trained model to disk

In [8]:
nlp.to_disk('./ner_model')

# Part 2

## Load trained model from disk

In [9]:
nlp = spacy.load('./ner_model')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")

## Define custom rules for EntityRuler

In [12]:
from pandas import *

# stockcodes.csv
# https://drive.google.com/file/d/1LSAu7fJkNXHqsSbPQmBMc-T3hUQaVO4a/view?usp=sharing
data = read_csv("stockcodes.csv",dtype=str) 

stockcodes = data.iloc[:,1].to_list()   # STOCK
orgs = data.iloc[:,2].to_list()         # ORG-S

org_alias = ['5號仔','大笨象','中女']     # ORG-S-A

print(stockcodes[:10])
print(orgs[:10])

['2', '3', '4', '5', '6', '8', '10', '11', '12', '14']
['中電控股', '香港中華煤氣', '九龍倉集團', '匯豐控股', '電能實業', '電訊盈科', '恒隆集團', '恒生銀行', '恒基地產', '希慎興業']


In [13]:
patterns = [
    {"label": "STOCK", "pattern": [{"IS_DIGIT": True},{"ORTH":"."},{"ORTH": "HK"}]},
    {"label": "STOCK", "pattern": [{"TEXT": {"REGEX":"\d+\.HK"}}]},
    {"label": "STOCK", "pattern": [{"ORTH": "("},{"IS_DIGIT": True},{"ORTH": ")"}]},
    {"label": "STOCK3", "pattern": [{'TEXT': {'IN': stockcodes}}]},
    {"label": "ORG-S", "pattern": [{'ORTH': {'IN': orgs}}]},
    {"label": "ORG-S-A", "pattern": [{'ORTH': {'IN': org_alias}}]},
    {"label": "STOCK", "pattern": [{"IS_DIGIT": True},{"ENT_TYPE": "ORG"}]},
    {"label": "STOCK2", "pattern": [{"POS": "NUM"},{"POS": "NOUN"}]},
]

if "entity_ruler" in nlp.pipe_names:
  nlp.remove_pipe("entity_ruler")

entity_ruler = nlp.add_pipe("entity_ruler", before='ner')
entity_ruler.add_patterns(patterns)

## Check stockcode suffix (entity label 'STOCK3')

In [29]:
from spacy.language import Language
from spacy.tokens import Span

def print_ent_details(ents):
    ent_details = []
    for ent in doc.ents:
      ent_details.append([ent.text, ent.label_])
    print('doc.ents: {}'.format(ent_details))

@Language.component("check_stockno_suffix")
def remove_stockno_suffix(doc):
    # print_ent_details(doc.ents)
    new_ents = []
    for i in range(len(doc.ents)):
      if doc.ents[i].label_ == "STOCK3":
        next_token = doc[doc.ents[i].start+1]
        # print('[{},{},{}]'.format(doc.ents[i].text,doc.ents[i].label_,next_token.text))
        if next_token.text != "元":
          new_ent = Span(doc, doc.ents[i].start, doc.ents[i].end, label="STOCK")
          new_ents.append(new_ent)
      else:
        new_ents.append(doc.ents[i])
    doc.ents = new_ents
    # print_ent_details(doc.ents)
    return doc

# Add the component after the named entity recognizer
if "check_stockno_suffix" in nlp.pipe_names:
  nlp.remove_pipe("check_stockno_suffix")

nlp.add_pipe("check_stockno_suffix", after="ner")

<function __main__.remove_stockno_suffix>

## Remove the noun suffix (entity label 'STOCK2')

In [30]:
from spacy.language import Language
from spacy.tokens import Span

@Language.component("remove_stockno_suffix")
def remove_stockno_suffix(doc):
    new_ents = []
    for ent in doc.ents:
        if ent.label_ == "STOCK2":  #and ent.start != 0:
            next_token = doc[ent.start + 1]
            if next_token.pos_ == "NOUN":
                new_ent = Span(doc, ent.start, ent.end - 1, label="STOCK")
                new_ents.append(new_ent)
        else:
            new_ents.append(ent)
    doc.ents = new_ents
    return doc

# Add the component after the named entity recognizer
if "remove_stockno_suffix" in nlp.pipe_names:
  nlp.remove_pipe("remove_stockno_suffix")

nlp.add_pipe("remove_stockno_suffix", after="ner")

<function __main__.remove_stockno_suffix>

# Test the trained NER model and EntityRuler

In [31]:
sentences = [
  "睇好港鐡(0066.HK)強烈建議買入 😎",
  "內地疫情受控,旅遊相關股同程藝龍(780)最近區間上落橫行,大戶收集似近完成,中線支持位$13.83,可考慮作中長線投資。",
  "睿見教育（6068.HK）：凈利與現金流大增超四成，估值吸引力凸顯，獲大行睇多	隻1765都賺多幾成 升左少少就跌凸 支股即係支股  🤷 ",
  "密切留意鋼鐵股，581中國東方，323馬鋼，347鞍鋼，1053重鋼，2600中鋁！",
  '入左中女 (308)  升四成，放定加馬	all in pk 左,起唔翻身  加一注 算, 3元走人!  贏咗購買 tesla 電車  輸咗就pk'
]

# def replace_zh_punctuation(sentence):
#   str = sentence
#   str = str.replace("（", "(")
#   str = str.replace("）", ")")
#   return(str)

for sentence in sentences:
  # sentence = replace_zh_punctuation(sentence)
  doc = nlp(sentence)
  displacy.render(doc,style='ent',jupyter=True)